In [1]:
!pip install nltk
!pip install wordcloud
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Libraries for visualisation
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#Libraries for formattting and handling text 
import string 
import re
#Library for nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS
#Library for Splitting Dataset
from sklearn.model_selection import train_test_split
#Libraries for NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Bidirectional, Dense, Dropout, Embedding, LSTM, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#Library for evaluation
from sklearn import metrics
from functools import reduce
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

2025-04-12 16:43:52.314218: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-12 16:43:52.324435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744447432.335527 2379575 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744447432.338829 2379575 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744447432.347772 2379575 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
train_data = pd.read_csv('/home/aicv/work/datasets/train.csv')

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def toclean_text(text):

    
    clean_text = [char for char in text if char not in string.punctuation]
   
    clean_text = ''.join(clean_text)
    
        
    return clean_text

In [6]:
train_data['clean_text'] = train_data['text'].apply(toclean_text)

In [7]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [8]:
# Remove all URLs, replace by URL
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)

# Remove HTML beacon
def remove_HTML(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Remove non printable characters
def remove_not_ASCII(text):
    text = ''.join([word for word in text if word in string.printable])
    return text

# Change an abbreviation by its true meaning
def word_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

# Replace all abbreviations
def replace_abbrev(text):
    string = ""
    for word in text.split():
        string += word_abbrev(word) + " "        
    return string

# Remove @ and mention, replace by USER
def remove_mention(text):
    at=re.compile(r'@\S+')
    return at.sub(r'USER',text)

# Remove numbers, replace it by NUMBER
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)

# Remove all emojis, replace by EMOJI
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'EMOJI', text)

# Replace some others smileys with SADFACE
def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

# Replace some smileys with SMILE
def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    #smiley = re.compile(r'#{eyes}#{nose}[)d]+|[)d]+#{nose}#{eyes}/i')
    return smiley.sub(r'SMILE', text)

# Replace <3 with HEART
def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)

In [9]:
def clean_tweet(text):
    
    # Remove non text
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_not_ASCII(text)
    
    # replace abbreviations, @ and number
    text = replace_abbrev(text)  
    text = remove_mention(text)
    text = remove_number(text)
    
    # Remove emojis / smileys
    text = remove_emoji(text)
    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
  
    return text

train_data["clean_text"] = train_data["clean_text"].apply(clean_tweet)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/aicv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
def toremove_stopword(text):
    remove_stopword = [word for word in text.split() if word.lower() not in stopwords.words('english')]

    return remove_stopword

train_data['clean_text'] = train_data['clean_text'].apply(toremove_stopword)

In [12]:
max_features=3000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data['clean_text'].values)
X = tokenizer.texts_to_sequences(train_data['clean_text'].values)
X = pad_sequences(X)

print(X.shape)

(7613, 20)


In [13]:
tokenizer.sequences_to_texts([[ 713,  154,   56, 1434,   14]])

['reason earthquake may allah us']

In [14]:
y = train_data['target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =41)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [15]:
# Hyperparameters
embed_dim = 1024
gru_units = 512  # Using GRU units

# Build the GRU based model
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(Dropout(0.2))
model.add(GRU(gru_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1))  # Final output layer for binary classification

# Compile the model
adam = optimizers.Adam(learning_rate=1e-4)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=adam, metrics=['accuracy'])

# Display the model summary
model.summary(expand_nested=True, show_trainable=True)

I0000 00:00:1744447437.950913 2379575 service.cc:152] XLA service 0x5ca68f1e4c20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744447437.950939 2379575 service.cc:160]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1744447438.011531 2379575 service.cc:152] XLA service 0x5ca68dd48ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744447438.011548 2379575 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1744447438.016739 2379575 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 810 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ embedding (Embedding)       │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ ?                     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ gru (GRU)                   │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ ?                     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, threshold, patience=2):
        super(CustomEarlyStopping, self).__init__()
        self.threshold = threshold
        self.patience = patience
        self.best_val_accuracy = 0.0
        self.wait_count = 0

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_val_acc = logs.get("val_accuracy")
        if current_val_acc is None:
            return

        # Check if we've reached the threshold
        if current_val_acc >= self.threshold:
            print(f"\n驗證準確率達到閾值 {self.threshold:.2f}，停止訓練！")
            self.model.stop_training = True
            return

        # If current accuracy is lower than best, increase wait count
        if current_val_acc < self.best_val_accuracy:
            self.wait_count += 1
            print(f"\n驗證準確率下降 {self.wait_count} epoch(s)")
            if self.wait_count >= self.patience:
                print(f"\n連續 {self.patience} 個 epoch 驗證準確率下降，停止訓練！")
                self.model.stop_training = True
        else:
            # Reset wait count if improvement is observed
            self.wait_count = 0

        # Update best validation accuracy if current is better
        if current_val_acc > self.best_val_accuracy:
            self.best_val_accuracy = current_val_acc

In [17]:
checkpoint = ModelCheckpoint('best_model.h5',   # 儲存路徑與檔名
                             monitor='val_accuracy',  # 監控驗證準確率
                             mode='max',              # 越大越好
                             save_best_only=True,     # 只儲存最佳模型
                             verbose=1)            

# 設定目標準確率為0.83
custom_early_stop = CustomEarlyStopping(threshold=0.90, patience=3)

model.fit(X_train, y_train, epochs = 50, batch_size=512, validation_data=(X_test, y_test),callbacks=[custom_early_stop, checkpoint])
model.save('final_model.h5')

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5769 - loss: 0.6870
Epoch 1: val_accuracy improved from -inf to 0.56927, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - accuracy: 0.5765 - loss: 0.6866 - val_accuracy: 0.5693 - val_loss: 0.6670
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5713 - loss: 0.6615
Epoch 2: val_accuracy did not improve from 0.56927
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5712 - loss: 0.6611 - val_accuracy: 0.5693 - val_loss: 0.6498
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5640 - loss: 0.6450
Epoch 3: val_accuracy did not improve from 0.56927
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5645 - loss: 0.6447 - val_accuracy: 0.5693 - val_loss: 0.6337
Epoch 4/50
11/12 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5752 - loss: 0.6222
Epoch 4: val_accuracy improved from 0.56927 to 0.58372, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.5749 - loss: 0.6215 - val_accuracy: 0.5837 - val_loss: 0.6095
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6021 - loss: 0.5946
Epoch 5: val_accuracy improved from 0.58372 to 0.66251, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.6032 - loss: 0.5937 - val_accuracy: 0.6625 - val_loss: 0.5719
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6858 - loss: 0.5396
Epoch 6: val_accuracy improved from 0.66251 to 0.74261, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.6872 - loss: 0.5385 - val_accuracy: 0.7426 - val_loss: 0.5103
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7785 - loss: 0.4618
Epoch 7: val_accuracy improved from 0.74261 to 0.80433, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.7788 - loss: 0.4604 - val_accuracy: 0.8043 - val_loss: 0.4550
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8181 - loss: 0.3983
驗證準確率下降 1 epoch(s)

Epoch 8: val_accuracy did not improve from 0.80433
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.8179 - loss: 0.3979 - val_accuracy: 0.7991 - val_loss: 0.4391
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8313 - loss: 0.3606
Epoch 9: val_accuracy improved from 0.80433 to 0.81221, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.8312 - loss: 0.3609 - val_accuracy: 0.8122 - val_loss: 0.4354
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8461 - loss: 0.3354
Epoch 10: val_accuracy improved from 0.81221 to 0.81550, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8459 - loss: 0.3359 - val_accuracy: 0.8155 - val_loss: 0.4384
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8621 - loss: 0.3191
驗證準確率下降 1 epoch(s)

Epoch 11: val_accuracy did not improve from 0.81550
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8616 - loss: 0.3195 - val_accuracy: 0.8148 - val_loss: 0.4401
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8560 - loss: 0.3179
驗證準確率下降 2 epoch(s)

Epoch 12: val_accuracy did not improve from 0.81550
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.8564 - loss: 0.3170 - val_accuracy: 0.8122 - val_loss: 0.4453
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8680 - loss: 0.2916
Epoch 13: val_accuracy improved from 0.81550 to 0.81812, saving model to best_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.8681 - loss: 0.2917 - val_accuracy: 0.8181 - val_loss: 0.4573
Epoch 14/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8764 - loss: 0.2760
驗證準確率下降 1 epoch(s)

Epoch 14: val_accuracy did not improve from 0.81812
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8764 - loss: 0.2763 - val_accuracy: 0.8129 - val_loss: 0.4624
Epoch 15/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8823 - loss: 0.2747
驗證準確率下降 2 epoch(s)

Epoch 15: val_accuracy did not improve from 0.81812
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8826 - loss: 0.2744 - val_accuracy: 0.8129 - val_loss: 0.4741
Epoch 16/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8848 - loss: 0.2614
驗證準確率下降 3 epoch(s)

連續 3 個 epoch 驗證準確率下降，停止訓練！

Epoch 16: val_accuracy did not improve from 0.81812
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8848 - loss: 0.2613 - val_accuracy: 0.8056 - val_loss: 0.4877


In [18]:
model = tf.keras.models.load_model('/home/aicv/work/best_model.h5')

In [19]:
y_pred = model.predict(X_test).round()
y_pred

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


array([[-3.],
       [ 3.],
       [ 3.],
       ...,
       [-1.],
       [-2.],
       [-2.]], dtype=float32)

In [20]:
logits = model.predict(X_train)
probabilities = 1 / (1 + np.exp(-logits))
y_train_pred = np.round(probabilities).astype(int)

train_accuracy_manual = metrics.accuracy_score(y_train, y_train_pred)
train_precision_manual = metrics.precision_score(y_train, y_train_pred)
train_recall_manual = metrics.recall_score(y_train, y_train_pred)

print("Train Accuracy:", train_accuracy_manual)
print("Train Precision:", train_precision_manual)
print("Train Recall:", train_recall_manual)

191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Train Accuracy: 0.8878489326765189
Train Precision: 0.90016570008285
Train Recall: 0.8309751434034417


In [21]:
y_pred_proba = 1/(1 + np.exp(-y_pred))  # Convert logits to probabilities
y_pred_binary = (y_pred_proba > 0.5).astype(int)

# Or if your predictions are already probabilities
# y_pred_binary = (y_pred > 0.5).astype(int)

print('Accuracy is: ', metrics.accuracy_score(y_test, y_pred_binary))
print('Recall is: ', metrics.recall_score(y_test, y_pred_binary))
print('Precision is: ', metrics.precision_score(y_test, y_pred_binary))

Accuracy is:  0.8181221273801708
Recall is:  0.7088414634146342
Precision is:  0.8439201451905626


In [22]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test data
test_data = pd.read_csv('/home/aicv/work/datasets/test.csv')

# Preprocess the text column using the same functions as in training
test_data['clean_text'] = test_data['text'].apply(toclean_text)
test_data['clean_text'] = test_data['clean_text'].apply(clean_tweet)
test_data['clean_text'] = test_data['clean_text'].apply(toremove_stopword)

# Convert text to sequences using the same tokenizer as during training
X_test = tokenizer.texts_to_sequences(test_data['clean_text'].values)

# Pad the sequences to the same length as during training
X_test = pad_sequences(X_test, maxlen=X.shape[1])

# Predict logits
logits = model.predict(X_test)

# Convert logits to probabilities
probabilities = 1 / (1 + np.exp(-logits))

# Round to get binary predictions
y_pred = np.round(probabilities).astype(int)
# Create the output in the required format (id and target)
submission = pd.DataFrame({
    'id': test_data['id'],  # Use the 'id' column from the test data
    'target': y_pred.flatten()  # Flatten predictions to match the format
})

# Save the predictions to a CSV file
submission.to_csv('/home/aicv/work/GRU_result.csv', index=False)

print("Predictions saved to 'GRU_result.csv'")

102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Predictions saved to 'GRU_result.csv'
